In [1]:
!pip install bayesian-optimization

#### 그리드, 랜덤, 베이지안 서치 모델

In [2]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import RandomizedSearchCV

#그 외 필요한 모듈 임포트
#from sklearn.metrics import make_scorer
#from sklearn.model_selection import cross_validate

ImportError: DLL load failed while importing _arpack: 지정된 프로시저를 찾을 수 없습니다.

In [3]:
import os
from os.path import join
import copy
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sklearn

import matplotlib.pyplot as plt

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from IPython.display import Image
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import roc_curve, auc, average_precision_score
from sklearn.metrics import r2_score

ImportError: DLL load failed while importing _arpack: 지정된 프로시저를 찾을 수 없습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler

#### model import

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
!pip install xgboost
from xgboost import plot_importance
from xgboost import XGBClassifier
!pip install lightgbm
from lightgbm import LGBMClassifier
!pip install catboost
from catboost import CatBoostClassifier

In [ ]:
data  = pd.read_csv("C:/Users/song/Downloads/BankChurners.csv (1)-20210223T041827Z-001/encoding_finished_v2.csv")

In [ ]:
dropped = data.drop('Unnamed: 0',axis=1)
dropped 

In [ ]:
x = dropped.drop('Attrition_Flag_Attrited Customer',axis=1)
x

In [ ]:
y = dropped['Attrition_Flag_Attrited Customer']
y

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=42)

## 1.Gradient boosting

In [ ]:
gbc = GradientBoostingClassifier(random_state=0,learning_rate=0.5)
gbc.fit(x_train,y_train)
y_1pred=gbc.predict(x_test)
y_1proba=gbc.predict_proba(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,y_1pred))
print('정확도: ',accuracy_score(y_test, y_1pred))
print('재현율: ',recall_score(y_test, y_1pred))
print('정밀도: ',precision_score(y_test, y_1pred))
print('f1점수: ',f1_score(y_test, y_1pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_1pred))

# train set 정확도
score_train = gbc.score(x_train, y_train) 
print('{:.3f}'.format(score_train))

# 일반화 정확도
score_test = gbc.score(x_test, y_test) 
print('{:.3f}'.format(score_test))

### 1-1. 랜더마이즈

In [ ]:
gbc = GradientBoostingClassifier()

dists = {
    'loss' : ['deviance', 'exponential'],
    #실수의 경우 보통은 아래처럼 10단위로 지정 
    'learning_rate' : [0.1, 0.01, 0.001, 0.0001], #[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55], 
    'n_estimators' : [100, 200, 300, 400, 500, 600],
    'max_depth':[2, 3, 4, 5,6,7,8,9,10]
}

rds = RandomizedSearchCV(
    gbc, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='f1',  #평가 기준: f1
    verbose=1,
    n_jobs=-1,
    random_state=0
)
rds.fit(x_train, y_train)
print('최적 하이퍼파라미터: ', rds.best_params_)

In [ ]:
pipe = rds.best_estimator_

y_val_pred = pipe.predict(x_test)
y_train_pred = pipe.predict(x_train)

print('훈련 f1 score: ', f1_score(y_train, y_train_pred, average='binary'))
print('검증 f1 score: ', f1_score(y_test, y_val_pred, average='binary'))
print('             분류 보고서')
print(classification_report(y_test,y_val_pred))
print('정확도: ',accuracy_score(y_test, y_val_pred))
print('재현율: ',recall_score(y_test, y_val_pred))
print('정밀도: ',precision_score(y_test, y_val_pred))
print('f1점수: ',f1_score(y_test, y_val_pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_pred))

### 1-2 베이지안

In [ ]:
def gbc_cv(learning_rate, n_estimators, max_depth):
    model = GradientBoostingClassifier(
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                max_depth = int(round(max_depth)),
                               )
    # 모델 훈련
    model.fit(x_train, y_train)

    # 예측값 출력
    bo_y_pred2= model.predict(x_test)

    # 각종 metric 계산
    acc = accuracy_score(y_test, bo_y_pred2)
    recall = recall_score(y_test, bo_y_pred2)
    precision= precision_score(y_test, bo_y_pred2)
    f1 = f1_score(y_test, bo_y_pred2)
    roc_auc = roc_auc_score(y_test, bo_y_pred2)
    
    # 사용할 metric 반환
    return f1

In [ ]:
# 실험해보고자하는 hyperparameter 집합
pbounds = {'learning_rate' : (0.0001, 0.1),
           'n_estimators': (100, 600),
           'max_depth': (1, 10),
          }

In [ ]:

bo_gbc = BayesianOptimization(f = gbc_cv, pbounds = pbounds, verbose = 2, random_state = 0 )
bo_gbc.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

In [ ]:
print(bo_gbc.max)

In [ ]:
#파라미터 적용
fit_gbc = GradientBoostingClassifier(learning_rate=bo_gbc.max['params']['learning_rate'],
                               n_estimators=int(bo_gbc.max['params']['n_estimators']),                                         
                               max_depth = int(round(bo_gbc.max['params']['max_depth'])),
                               )

fit_gbc.fit(x_train, y_train)
bo_gbc_ypred=fit_gbc.predict(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,bo_gbc_ypred))
print('정확도: ',accuracy_score(y_test, bo_gbc_ypred))
print('재현율: ',recall_score(y_test, bo_gbc_ypred))
print('정밀도: ',precision_score(y_test, bo_gbc_ypred))
print('f1점수: ',f1_score(y_test, bo_gbc_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_gbc_ypred))

In [ ]:
# train set 정확도
score_train = fit_gbc.score(x_train, y_train) 
print('{:.3f}'.format(score_train))

# 일반화 정확도
score_test = fit_gbc.score(x_test, y_test) 
print('{:.3f}'.format(score_test))

### 1-3 디폴트,랜덤서치,베이지안서치 점수비교(그래디언트 부스팅)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,y_1pred))
print('정확도: ',accuracy_score(y_test, y_1pred))
print('재현율: ',recall_score(y_test, y_1pred))
print('정밀도: ',precision_score(y_test, y_1pred))
print('f1점수: ',f1_score(y_test, y_1pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_1pred))

print('훈련 f1 score: ', f1_score(y_train, y_train_pred, average='binary'))
print('검증 f1 score: ', f1_score(y_test, y_val_pred, average='binary'))
print('             분류 보고서')
print(classification_report(y_test,y_val_pred))
print('정확도: ',accuracy_score(y_test, y_val_pred))
print('재현율: ',recall_score(y_test, y_val_pred))
print('정밀도: ',precision_score(y_test, y_val_pred))
print('f1점수: ',f1_score(y_test, y_val_pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_pred))

print('             분류 보고서')
print(classification_report(y_test,bo_gbc_ypred))
print('정확도: ',accuracy_score(y_test, bo_gbc_ypred))
print('재현율: ',recall_score(y_test, bo_gbc_ypred))
print('정밀도: ',precision_score(y_test, bo_gbc_ypred))
print('f1점수: ',f1_score(y_test, bo_gbc_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_gbc_ypred))

# 베이지안이 가장 좋음


## 2.xgboosting

In [ ]:
xgb = XGBClassifier(n_estimators=500, learning_rate=1.0, max_depth =4)
xgb.fit(x_train, y_train)
y_2pred = xgb.predict(x_test)
y_2proba = xgb.predict_proba(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_2pred))
print('정확도: ',accuracy_score(y_test, y_2pred)) 
print('재현율: ',recall_score(y_test,  y_2pred))
print('정밀도: ',precision_score(y_test,  y_2pred))
print('f1점수: ',f1_score(y_test,  y_2pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_2pred))

### 2-1 랜덤서치

In [ ]:
xgb = XGBClassifier()

dists = {
    'loss' : ['deviance', 'exponential'],
    #실수의 경우 보통은 아래처럼 10단위로 지정 
    'learning_rate' : [0.1, 0.01, 0.001, 0.0001], #[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55], 
    'n_estimators' : [100, 200, 300, 400, 500, 600],
    'max_depth':[2, 3, 4, 5,6,7,8,9,10]
}

rds1 = RandomizedSearchCV(
    xgb, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='f1',  #평가 기준: f1
    verbose=1,
    n_jobs=-1,
    random_state=0
)
rds1.fit(x_train, y_train)
print('최적 하이퍼파라미터: ', rds1.best_params_)

In [ ]:
pipe2 = rds1.best_estimator_

y_val_2pred = pipe2.predict(x_test)
y_train_2pred = pipe2.predict(x_train)

print('훈련 f1 score: ', f1_score(y_train, y_train_2pred, average='binary'))
print('검증 f1 score: ', f1_score(y_test, y_val_2pred, average='binary'))
print('             분류 보고서')
print(classification_report(y_test,y_val_2pred))
print('정확도: ',accuracy_score(y_test, y_val_2pred))
print('재현율: ',recall_score(y_test, y_val_2pred))
print('정밀도: ',precision_score(y_test, y_val_2pred))
print('f1점수: ',f1_score(y_test, y_val_2pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_2pred))

### 2-2 베이지안

In [ ]:
from xgboost import XGBClassifier
# 모델 정의
def xgb_cv(learning_rate, n_estimators, max_depth):
    model1 = XGBClassifier(
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                max_depth = int(round(max_depth)),
                               )
    # 모델 훈련
    model1.fit(x_train, y_train)

    # 예측값 출력
    bo_y_pred3= model1.predict(x_test)

    # 각종 metric 계산
    acc = accuracy_score(y_test, bo_y_pred3)
    recall = recall_score(y_test, bo_y_pred3)
    precision= precision_score(y_test, bo_y_pred3)
    f1 = f1_score(y_test, bo_y_pred3)
    roc_auc = roc_auc_score(y_test, bo_y_pred3)
    
    # 사용할 metric 반환
    return f1

In [ ]:
bo_xgb = BayesianOptimization(f = xgb_cv, pbounds = pbounds, verbose = 2, random_state = 0 )
bo_xgb.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

In [ ]:
print(bo_xgb.max)

In [ ]:
#파라미터 적용
fit_xgb = XGBClassifier(learning_rate=bo_xgb.max['params']['learning_rate'],
                               n_estimators=int(bo_xgb.max['params']['n_estimators']),                                         
                               max_depth = int(round(bo_xgb.max['params']['max_depth'])),
                               )

fit_xgb.fit(x_train, y_train)
bo_xgb_ypred=fit_gbc.predict(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,bo_xgb_ypred))
print('정확도: ',accuracy_score(y_test, bo_xgb_ypred))
print('재현율: ',recall_score(y_test, bo_xgb_ypred))
print('정밀도: ',precision_score(y_test, bo_xgb_ypred))
print('f1점수: ',f1_score(y_test, bo_xgb_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_xgb_ypred))

### 2-3 디폴트,랜덤서치,베이지안서치 점수비교(그래디언트 부스팅)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_2pred))
print('정확도: ',accuracy_score(y_test, y_2pred)) 
print('재현율: ',recall_score(y_test,  y_2pred))
print('정밀도: ',precision_score(y_test,  y_2pred))
print('f1점수: ',f1_score(y_test,  y_2pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_2pred))

print('             분류 보고서')
print(classification_report(y_test,y_val_2pred))
print('정확도: ',accuracy_score(y_test, y_val_2pred))
print('재현율: ',recall_score(y_test, y_val_2pred))
print('정밀도: ',precision_score(y_test, y_val_2pred))
print('f1점수: ',f1_score(y_test, y_val_2pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_2pred))

print('             분류 보고서')
print(classification_report(y_test,bo_xgb_ypred))
print('정확도: ',accuracy_score(y_test, bo_xgb_ypred))
print('재현율: ',recall_score(y_test, bo_xgb_ypred))
print('정밀도: ',precision_score(y_test, bo_xgb_ypred))
print('f1점수: ',f1_score(y_test, bo_xgb_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_xgb_ypred))

## 3.LightGBM

In [ ]:
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(x_train, y_train)
y_3pred = lgbm.predict(x_test)
y_3proba = lgbm.predict_proba(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_3pred))
print('정확도: ',accuracy_score(y_test, y_3pred)) 
print('재현율: ',recall_score(y_test,  y_3pred))
print('정밀도: ',precision_score(y_test,  y_3pred))
print('f1점수: ',f1_score(y_test,  y_3pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_3pred))

### 3-1 랜덤서치

In [ ]:
lgbm = LGBMClassifier()

dists = {
    'loss' : ['deviance', 'exponential'],
    #실수의 경우 보통은 아래처럼 10단위로 지정 
    'learning_rate' : [0.1, 0.01, 0.001, 0.0001], #[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55], 
    'n_estimators' : [100, 200, 300, 400, 500, 600],
    'max_depth':[2, 3, 4, 5,6,7,8,9,10]
}

rds2 = RandomizedSearchCV(
    lgbm, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='f1',  #평가 기준: f1
    verbose=1,
    n_jobs=-1,
    random_state=0
)
rds2.fit(x_train, y_train)
print('최적 하이퍼파라미터: ', rds2.best_params_)

In [ ]:
pipe3 = rds2.best_estimator_

y_val_3pred = pipe3.predict(x_test)
y_train_3pred = pipe3.predict(x_train)

print('훈련 f1 score: ', f1_score(y_train, y_train_3pred, average='binary'))
print('검증 f1 score: ', f1_score(y_test, y_val_3pred, average='binary'))
print('             분류 보고서')
print(classification_report(y_test,y_val_3pred))
print('정확도: ',accuracy_score(y_test, y_val_3pred))
print('재현율: ',recall_score(y_test, y_val_3pred))
print('정밀도: ',precision_score(y_test, y_val_3pred))
print('f1점수: ',f1_score(y_test, y_val_3pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_3pred))

### 3-2 베이지안

In [ ]:
# 모델 정의
def lgbm_cv(learning_rate, n_estimators, max_depth):
    model2 = LGBMClassifier(
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                max_depth = int(round(max_depth)),
                               )
    # 모델 훈련
    model2.fit(x_train, y_train)

    # 예측값 출력
    bo_y_pred4= model2.predict(x_test)

    # 각종 metric 계산
    acc = accuracy_score(y_test, bo_y_pred4)
    recall = recall_score(y_test, bo_y_pred4)
    precision= precision_score(y_test, bo_y_pred4)
    f1 = f1_score(y_test, bo_y_pred4)
    roc_auc = roc_auc_score(y_test, bo_y_pred4)
    
    # 사용할 metric 반환
    return f1

In [ ]:
bo_lgbm = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 0 )
bo_lgbm.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

In [ ]:
print(bo_lgbm.max)

In [ ]:
#파라미터 적용
fit_lgbm = LGBMClassifier(learning_rate=bo_lgbm.max['params']['learning_rate'],
                               n_estimators=int(bo_lgbm.max['params']['n_estimators']),                                         
                               max_depth = int(round(bo_lgbm.max['params']['max_depth'])),
                               )

fit_lgbm.fit(x_train, y_train)
bo_lgbm_ypred=fit_lgbm.predict(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,bo_lgbm_ypred))
print('정확도: ',accuracy_score(y_test, bo_lgbm_ypred))
print('재현율: ',recall_score(y_test, bo_lgbm_ypred))
print('정밀도: ',precision_score(y_test, bo_lgbm_ypred))
print('f1점수: ',f1_score(y_test, bo_lgbm_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_lgbm_ypred))

### 3-3 디폴트,랜덤서치,베이지안서치 점수비교(그래디언트 부스팅)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_3pred))
print('정확도: ',accuracy_score(y_test, y_3pred)) 
print('재현율: ',recall_score(y_test,  y_3pred))
print('정밀도: ',precision_score(y_test,  y_3pred))
print('f1점수: ',f1_score(y_test,  y_3pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_3pred))

print('             분류 보고서')
print(classification_report(y_test,y_val_3pred))
print('정확도: ',accuracy_score(y_test, y_val_3pred))
print('재현율: ',recall_score(y_test, y_val_3pred))
print('정밀도: ',precision_score(y_test, y_val_3pred))
print('f1점수: ',f1_score(y_test, y_val_3pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_3pred))

print('             분류 보고서')
print(classification_report(y_test,bo_lgbm_ypred))
print('정확도: ',accuracy_score(y_test, bo_lgbm_ypred))
print('재현율: ',recall_score(y_test, bo_lgbm_ypred))
print('정밀도: ',precision_score(y_test, bo_lgbm_ypred))
print('f1점수: ',f1_score(y_test, bo_lgbm_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_lgbm_ypred))

## 4. catboost

In [ ]:
cbc = CatBoostClassifier()
cbc.fit(x_train, y_train)
y_4pred = cbc.predict(x_test)
y_4proba = cbc.predict_proba(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_4pred))
print('정확도: ',accuracy_score(y_test, y_4pred)) 
print('재현율: ',recall_score(y_test,  y_4pred))
print('정밀도: ',precision_score(y_test,  y_4pred))
print('f1점수: ',f1_score(y_test,  y_4pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_4pred))

### 4-1랜덤서치

In [ ]:
cbc = CatBoostClassifier()

dists = {
    #'loss' : ['deviance', 'exponential'],
    #실수의 경우 보통은 아래처럼 10단위로 지정 
    'learning_rate' : [0.1, 0.01, 0.001, 0.0001], #[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55], 
    'n_estimators' : [100, 200, 300, 400, 500, 600],
    'max_depth':[2, 3, 4, 5,6,7,8,9,10]
}

rds3 = RandomizedSearchCV(
    cbc, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='f1',  #평가 기준: f1
    verbose=1,
    n_jobs=-1,
    random_state=0
)
rds3.fit(x_train, y_train)
print('최적 하이퍼파라미터: ', rds3.best_params_)

In [ ]:
pipe4 = rds3.best_estimator_

y_val_4pred = pipe4.predict(x_test)
y_train_4pred = pipe4.predict(x_train)

print('훈련 f1 score: ', f1_score(y_train, y_train_4pred, average='binary'))
print('검증 f1 score: ', f1_score(y_test, y_val_4pred, average='binary'))
print('             분류 보고서')
print(classification_report(y_test,y_val_4pred))
print('정확도: ',accuracy_score(y_test, y_val_4pred))
print('재현율: ',recall_score(y_test, y_val_4pred))
print('정밀도: ',precision_score(y_test, y_val_4pred))
print('f1점수: ',f1_score(y_test, y_val_4pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_4pred))

### 4-2 베이지안

In [ ]:
# 모델 정의
def cbc_cv(learning_rate, n_estimators, max_depth):
    model3 = CatBoostClassifier(
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                max_depth = int(round(max_depth)),
                               )
    # 모델 훈련
    model3.fit(x_train, y_train)

    # 예측값 출력
    bo_y_pred5= model3.predict(x_test)

    # 각종 metric 계산
    acc = accuracy_score(y_test, bo_y_pred5)
    recall = recall_score(y_test, bo_y_pred5)
    precision= precision_score(y_test, bo_y_pred5)
    f1 = f1_score(y_test, bo_y_pred5)
    roc_auc = roc_auc_score(y_test, bo_y_pred5)
    
    # 사용할 metric 반환
    return f1

In [ ]:
bo_cbc = BayesianOptimization(f = cbc_cv, pbounds = pbounds, verbose = 2, random_state = 0 )
bo_cbc.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)
print(bo_cbc.max)

In [ ]:
#파라미터 적용
fit_cbc = LGBMClassifier(learning_rate=bo_cbc.max['params']['learning_rate'],
                               n_estimators=int(bo_cbc.max['params']['n_estimators']),                                         
                               max_depth = int(round(bo_cbc.max['params']['max_depth'])),
                               )

fit_cbc.fit(x_train, y_train)
bo_cbc_ypred=fit_cbc.predict(x_test)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test,bo_cbc_ypred))
print('정확도: ',accuracy_score(y_test, bo_cbc_ypred))
print('재현율: ',recall_score(y_test, bo_cbc_ypred))
print('정밀도: ',precision_score(y_test, bo_cbc_ypred))
print('f1점수: ',f1_score(y_test, bo_cbc_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_cbc_ypred))

### 4-3 디폴트,랜덤서치,베이지안서치 점수비교(그래디언트 부스팅)

In [ ]:
print('             분류 보고서')
print(classification_report(y_test, y_4pred))
print('정확도: ',accuracy_score(y_test, y_4pred)) 
print('재현율: ',recall_score(y_test,  y_4pred))
print('정밀도: ',precision_score(y_test,  y_4pred))
print('f1점수: ',f1_score(y_test,  y_4pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_4pred))

print('             분류 보고서')
print(classification_report(y_test,y_val_4pred))
print('정확도: ',accuracy_score(y_test, y_val_4pred))
print('재현율: ',recall_score(y_test, y_val_4pred))
print('정밀도: ',precision_score(y_test, y_val_4pred))
print('f1점수: ',f1_score(y_test, y_val_4pred))
print('ROC-AUC점수: ',roc_auc_score(y_test, y_val_4pred))

print('             분류 보고서')
print(classification_report(y_test,bo_cbc_ypred))
print('정확도: ',accuracy_score(y_test, bo_cbc_ypred))
print('재현율: ',recall_score(y_test, bo_cbc_ypred))
print('정밀도: ',precision_score(y_test, bo_cbc_ypred))
print('f1점수: ',f1_score(y_test, bo_cbc_ypred))
print('ROC-AUC점수: ',roc_auc_score(y_test, bo_cbc_ypred))

## 5. 총 정리

In [ ]:
print('          오리지널 데이터')
Comparison_orginal_data = pd.DataFrame({"GradBoost":[accuracy_score(y_test, y_1pred),recall_score(y_test,  y_1pred),precision_score(y_test,  y_1pred),f1_score(y_test,  y_1pred),roc_auc_score(y_test, y_1pred)],
"XGBoost":[accuracy_score(y_test, y_2pred),recall_score(y_test,  y_2pred),precision_score(y_test,  y_2pred),f1_score(y_test,  y_2pred),roc_auc_score(y_test, y_2pred)],
"LightGBM":[accuracy_score(y_test, y_3pred),recall_score(y_test,  y_3pred),precision_score(y_test,  y_3pred),f1_score(y_test,  y_3pred),roc_auc_score(y_test, y_3pred)],
'CatBoost':[accuracy_score(y_test, y_4pred),recall_score(y_test,  y_4pred),precision_score(y_test,  y_4pred),f1_score(y_test,  y_4pred),roc_auc_score(y_test, y_4pred)]},
                          index = ['ACC', 'Recall', 'Precision', 'F1', 'ROC-AUC'])
print(Comparison_orginal_data)

In [ ]:
print('          랜덤 서치 데이터')
Comparison_random = pd.DataFrame({"GradBoost":[accuracy_score(y_test, y_val_pred),recall_score(y_test,  y_val_pred),precision_score(y_test,  y_val_pred),f1_score(y_test,  y_val_pred),roc_auc_score(y_test, y_val_pred)],
"XGBoost":[accuracy_score(y_test, y_val_2pred),recall_score(y_test,  y_val_2pred),precision_score(y_test,  y_val_2pred),f1_score(y_test,  y_val_2pred),roc_auc_score(y_test, y_val_2pred)],
"LightGBM":[accuracy_score(y_test, y_val_3pred),recall_score(y_test,  y_val_3pred),precision_score(y_test,  y_val_3pred),f1_score(y_test,  y_val_3pred),roc_auc_score(y_test, y_val_3pred)],
'CatBoost':[accuracy_score(y_test, y_val_4pred),recall_score(y_test,  y_val_4pred),precision_score(y_test,y_val_4pred),f1_score(y_test,  y_val_4pred),roc_auc_score(y_test, y_val_4pred)]},
                          index = ['ACC', 'Recall', 'Precision', 'F1', 'ROC-AUC'])
print(Comparison_random)

In [ ]:
print('          베이지안 서치 데이터')
Comparison_random = pd.DataFrame({"GradBoost":[accuracy_score(y_test, bo_gbc_ypred),recall_score(y_test,  bo_gbc_ypred),precision_score(y_test,  bo_gbc_ypred),f1_score(y_test,  bo_gbc_ypred),roc_auc_score(y_test, bo_gbc_ypred)],
"XGBoost":[accuracy_score(y_test, bo_xgb_ypred),recall_score(y_test,  bo_xgb_ypred),precision_score(y_test,  bo_xgb_ypred),f1_score(y_test,  bo_xgb_ypred),roc_auc_score(y_test, bo_xgb_ypred)],
"LightGBM":[accuracy_score(y_test, bo_lgbm_ypred),recall_score(y_test,  bo_lgbm_ypred),precision_score(y_test,  bo_lgbm_ypred),f1_score(y_test,  bo_lgbm_ypred),roc_auc_score(y_test, bo_lgbm_ypred)],
'CatBoost':[accuracy_score(y_test, bo_cbc_ypred),recall_score(y_test,  bo_cbc_ypred),precision_score(y_test,bo_cbc_ypred),f1_score(y_test,  bo_cbc_ypred),roc_auc_score(y_test, bo_cbc_ypred)]},
                          index = ['ACC', 'Recall', 'Precision', 'F1', 'ROC-AUC'])
print(Comparison_random)